In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_data = pd.read_csv("../kaggle/train.csv") # read csv
test_data = pd.read_csv("../kaggle/test.csv")

In [4]:
train_data=train_data.drop(columns=['id'])
train_data

,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul),label
0,3556.0,2489.0,265.19,77.53,176.55,0.00,4.20,307.91,52,0,7515.0,1
1,1906.0,134.0,1442.61,551.90,876.07,112.10,168.15,1735.48,20,1,1756.0,0
2,1586.0,71.0,1332.74,684.20,655.26,244.95,216.52,1820.04,28,1,1311.0,0
3,683.0,94.0,419.23,255.80,162.17,72.05,44.68,538.22,55,1,1443.0,0
4,1032.0,71.0,1102.72,480.27,625.30,188.78,130.77,1427.97,28,1,1542.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
82,626.0,68.0,1771.57,666.99,1117.48,360.21,118.84,2306.82,42,1,1521.0,0
83,1237.0,71.0,1348.53,428.09,924.69,120.02,48.67,1524.78,56,0,1345.0,0
84,634.0,1002.0,1300.00,558.00,724.00,67.00,105.00,1484.26,34,0,2926.0,1
85,112.0,884.0,942.83,378.49,567.06,116.77,31.81,1104.59,33,1,2352.0,1


In [5]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns
missing_values = missing_values_table(train_data)
missing_values

Your selected dataframe has 12 columns.
There are 3 columns that have missing values.


,Missing Values,% of Total Values
MO HLADR+ MFI (cells/ul),1,1.1
Neu CD64+MFI (cells/ul),1,1.1
Mono CD64+MFI (cells/ul),1,1.1


# fill the missing values

In [6]:
from sklearn.impute import KNNImputer
imputer=KNNImputer(n_neighbors=3)
train_filled=imputer.fit_transform(train_data)


# add features based on clinical literature

In [7]:
train_filled_df=pd.DataFrame(train_filled)

In [8]:
train_filled_df_y=train_data.iloc[:,11]
train_filled_df_x=train_filled_df.iloc[:,:11]

In [10]:
train_filled_df_x["ratio4/8"]=train_filled_df_x.iloc[:,4]/train_filled_df_x.iloc[:,3]
train_filled_df_x["ration3/HLADR"]=train_filled_df_x.iloc[:,2]/train_filled_df_x.iloc[:,0]


# Model Logistic regression & Decision Tree

In [12]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()
model1.fit(train_filled_df_x, train_filled_df_y)

model2 = LogisticRegression()
model2.fit(train_filled_df_x.iloc[:,:11], train_filled_df_y)

LogisticRegression()

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier()
model3.fit(train_filled_df_x, train_filled_df_y)

model4 = RandomForestClassifier()
model4.fit(train_filled_df_x.iloc[:,:11], train_filled_df_y)

RandomForestClassifier()

In [14]:
test_data=test_data.drop(columns=['id'])
test_data["ratio4/8"]=test_data.iloc[:,4]/test_data.iloc[:,3]
test_data["ration3/HLADR"]=test_data.iloc[:,2]/test_data.iloc[:,0]

In [16]:
from collections import Counter 

In [17]:
r1=model1.predict(test_data)
Counter(r1)

Counter({0: 47, 1: 12})

In [19]:
r3=model3.predict(test_data)
Counter(r3)

Counter({0: 46, 1: 13})

In [29]:
# choose the logistic regression
sample=pd.DataFrame(r1,columns=['label'])
sample.index.name = 'id'
sample.to_csv('sample.csv')